In [ ]:
# import torch
# print(f"\n🔍 PyTorch: {torch.__version__}")
# print(f"🔍 GPU: {torch.cuda.is_available()}")
# print(f"🔍 Nome da GPU: {torch.cuda.get_device_name(0)}")

# # Nova forma de verificar ROCm a partir do PyTorch 2.0+
# print(f"🔍 Backend GPU: {'ROCm' if torch.version.hip else 'CUDA'}")
# print(f"🔍 Versão HIP: {torch.version.hip}\n")

# Carregando Pacotes e Tokenizador:

In [ ]:
from transformers import pipeline
import torch
import pdfplumber

from transformers import AutoTokenizer


In [ ]:
# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

In [ ]:
# Carregar modelo BERTimbau pré-treinado para Q&A
qa_pipeline = pipeline(
    "question-answering",
    model="bert-large-uncased-whole-word-masking-finetuned-squad",  # Modelo em inglês
    tokenizer="bert-large-uncased-whole-word-masking-finetuned-squad",
    device=device
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Tratamento do Documento

In [ ]:
def carregar_documento(caminho_do_pdf):
    """Carrega o texto de um PDF, removendo espaços extras e páginas vazias."""
    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages 
            if page.extract_text()  # Ignora páginas sem texto
        )
        texto = " ".join(texto.split())  # Normaliza espaços
    return texto

In [ ]:
def dividir_em_chunks_tokenizados(texto):
    """Divide o texto em chunks tokenizados, respeitando o limite do modelo."""

    # tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
    chunk_size = 400
    overlap = 100

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

# Definição de Processamento das Perguntas e das Respostas

In [ ]:
def processar_pergunta(pergunta, documento):
    """
    Processa a pergunta sobre os chunks do documento e retorna a melhor resposta.
    
    Args:
        pergunta (str): Pergunta do usuário.
        chunks (list): Lista de chunks do documento (já tokenizados).
        qa_pipeline: Pipeline de Q&A carregado (Bloco 2).
        tokenizer: Tokenizador do modelo.
        min_score (float): Score mínimo para considerar uma resposta válida.
    
    Returns:
        dict: {"answer": str, "score": float, "context": str} ou None se nenhuma resposta for encontrada.
    """
    chunks = dividir_em_chunks_tokenizados(documento)
    
    # 1. Tokeniza a pergunta para verificar tamanho
    tokens_pergunta = tokenizer.tokenize(pergunta)
    if len(tokens_pergunta) > 60:  # Limite arbitrário (ajuste conforme necessário)
        print("Pergunta muito longa! Simplifique para melhor precisão.")
    
    # 2. Executa Q&A em cada chunk
    respostas = []
    for chunk in chunks:
        try:
            resposta = qa_pipeline(question=pergunta, context=chunk)
            respostas.append(resposta)

            resposta_completa = {
                'answer': resposta.get('answer', ''),
                'score': resposta.get('score', 0),
                'context': resposta.get('context', chunk[:500])  # Fallback: 500 primeiros chars
            }
            respostas.append(resposta_completa)
        except Exception as e:
            print(f"Erro no chunk: {str(e)}")
            continue
    
    # 3. Filtra respostas com score baixo e seleciona a melhor
    respostas_validas = [r for r in respostas if r['score'] >= 0.1]
    if not respostas_validas:
        print("Não há resposta sobre isto nesse documento.")
        return None
    
    return max(respostas_validas, key=lambda x: x['score'])

# Pergunta e Resposta

In [ ]:
#Carrega o documento em PDF

documento = carregar_documento("arquivo.pdf")

In [ ]:
#Pergunta sobre o documento
pergunta = "What is the main topic of this document?"

In [ ]:
#Resposta gerada
resposta = processar_pergunta(pergunta, documento)

In [ ]:
print(resposta)